<span style="color:black"><span style = "font-size:30px"> Acquisition of promoter dataset</span>
&nbsp;&nbsp;&nbsp;
   
    
From a previous differential RNA-sequencing (dRNA-seq) dataset, we obtained native promoter sequences of Synechocystis sp. PCC 6803. We assumed the 100-base sequences from a transcription start site (TSS) as a promoter and took normalized reads of the corresponding transcript in the dRNA-seq dataset as the promoter strength. Below is the code description for acquiring the promoter dataset.  

1.	To acquire the promoter dataset from the dRNA-seq dataset, install ‘numpy’ and ‘pandas’ modules, which are modules for handling data arrays and analyzing datasets. 

In [ ]:
# 1.

import numpy as np
import pandas as pd

2.	Obtain the whole genome sequence of Synechocystis sp. PCC 6803 in a long string format.

In [ ]:
# 2.

pccgene = []
pccseq = ''
data = open('Whole_genome_of_synechocystis.txt','r')
line = data.readlines()
gspoint = line.index('ORIGIN\n')
for i in range(len(line)-gspoint-1):
    templist = line[i+gspoint].strip().split()[1:]
    for partseq in templist:
        pccseq = pccseq+partseq
data.close()
pccseq = pccseq.upper()

3.	Extract the promoter dataset composed of promoter sequences and their strength from the whole genome sequence and the dRNA-seq dataset

In [ ]:
# 3.

trlist = []
comp_dic = {'A':'T','G':'C','C':'G','T':'A'}
prolen = 100
df = pd.read_excel('Differential_RNA_sequencing_data.xlsx',header = 1,sheet_name = 'S1 - All TUs')
numpydf = df.to_numpy()
for i in numpydf:
    if i[0] == 'Chr':
        if i[4] == '+':
            trlist.append((pccseq[i[2]-(prolen+1):i[2]-1],i[19]))
        else:
            compseq = ''
            seq = pccseq[i[2]:i[2]+prolen]
            for s in seq:
                compseq +=comp_dic[s]
            revcomp_seq = compseq[::-1]
            trlist.append((revcomp_seq, i[19]))
    else:
        break

4.	 Export the promoter dataset containing promoter sequences and their strength. This dataset is a training dataset for the VAE and CNN models. 

In [ ]:
# 4. 

from openpyxl import Workbook
write_wb = Workbook()
write_ws = write_wb.active

write_ws['A1'] = 'Promoter'
write_ws['B1'] = 'Reads'

for i in range(len(trlist)):
    write_ws.cell(i+2,1,trlist[i][0])
    write_ws.cell(i+2,2,trlist[i][1])

write_wb.save('Training dataset.xlsx')